In [ ]:
import torch
from efficientnet_pytorch import EfficientNet
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from pathlib import Path

In [ ]:
# you can find a pretrained model at model/b3.pth
MODEL_F = 'C:/Users/Cana/Documents/GitHub/pyslam/outputPITCH.pth'
# directory with the numpy optical flow images you want to use for inference
OF_NPY_DIR = 'C:/Users/Cana/Documents/GitHub/pyslam/outFlow0'
OP_DIR='C:/Users/Cana/Documents/GitHub/pyslam/predictsPITCH'

In [ ]:
# check if cuda is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Load Model

In [ ]:
V = 0     # what version of efficientnet did you use
IN_C = 2  # number of input channels
NUM_C = 1 # number of classes to predict

In [ ]:
model = EfficientNet.from_pretrained(f'efficientnet-b{V}', in_channels=IN_C, num_classes=NUM_C)
state = torch.load(MODEL_F)
model.load_state_dict(state.state_dict())
#model.load_state_dict(state)
model.to(device);

In [ ]:
def inference(of_f):
    of = np.load(of_f)
    i = torch.from_numpy(of).to(device)
    pred = model(i)
    del i
    torch.cuda.empty_cache()
    return pred

In [ ]:
# loop over all files in directory and predict
images = list(Path(OF_NPY_DIR).glob('*.npy'))

guesses = np.empty((1200))
#print("images",images)
useLbl = 0
for i in range(1,len(images)):
    useI = i % 1200
    print("i" + str(i),len(images)-1)
    if (useI == 0 or (useLbl == 4 and i == len(images)-1)) and i!=0: 
        
        np.savetxt(f"{OP_DIR}/{useLbl}.txt",guesses)
        print("SAVING TO " + str(useLbl))
        
        guesses = np.empty((1200))
        useLbl = useLbl + 1
    
    f = Path(f'{Path(OF_NPY_DIR)}/{i}.npy')
    
    y_hat = inference(f).item()
    guesses[useI] = y_hat
    print(f'{f.name}-{useLbl}: {round(y_hat, 15)}')